# Initial Config


Note from Mekael
- For my team, use a virutal environment to keep deployment operations clean and tidy
- You can do this by running the following command in your terminal
- `python -m venv error-in-translations`
- Activate the environment by selecting it as your kernel for your jupyter notebook. If it doesn't work you will have to figure it out
- pip install the packages in the requirements file in the root directory of this repo
- `pip install -r requirements.txt`


- **If you install new packages that are not included in the environment, please add it to the requirements file manually or generate a new requirements file with the following command in the terminal**
- `pip freeze > requirements.txt`

https://github.com/openlanguagedata/flores

In [45]:
# !pip install sacrebleu
# !pip install python-dotenv

You should consider upgrading via the 'C:\Users\mekae\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [17]:
import pandas as pd

import openai
from openai import OpenAI
import os
import spacy
import random
import sacrebleu

# Not working
OPENAI_API_KEY = 'sk-proj-o9TONJi0MW2tSiDMhRkxT3BlbkFJkUr03XQ5IfUaxamV0e3k'

# Mekael's Personal Key, not being shared
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Baseline Benchmark

This benchmark sets a baseline and tests the translation precision & accuracy our POC pipeline, against the bare translation capabilities of OPENAI's CHATGPT 3.5 Turbo via their API.

If our POC performs better than the stock GPT 3.5 Turbo, it means that our proposed method is valubale and worthwhile to implement. 

For baseline testing purposes, our POC makes use of custom GPT 3.5 API prompting as the translation model as well as the quality estimation model. These will be replaced with a more sophisticated custom LLM solution during actual implementation.

We will be using Meta's Flores 200 dataset for this testing, and scores will be in the form of spBLEU and/or chrF++

---

# Implementation

## View the dataset

In [3]:
english_dataset = "../flores/floresp-v2.0-rc.2/dev/dev.eng_Latn"

# Rename the Column
column_names = ["Text Lines"]

# Read in the Flores Dataset English Latn
df = pd.read_csv(english_dataset, delimiter = '\t', header=None, names=column_names)
# df

## Translations using our proposed method (POC) and stock GPT 3.5 Turbo

### Add POC (no glossary)

In [120]:
# prompt with entity translation
def prompt_generator(text, source_language, target_language):
  # terms = identify_entities(text)
  # terms_translations = '. '.join([f"'{k}': '{v}'" for k, v in terms.items()])
  # translations = f"Based on the translation of following term: {terms_translations}."
  prompt = f"Translate the following text from {source_language[8:]} into {target_language[8:]}: {text}\n"
  # if terms == {}:
    # return prompt
  # prompt = translations + prompt
  
  prompt = prompt

  return prompt

# prompt = prompt_generator('光纤照上去变成黑光纤了', 'zh', 'en')
# print(prompt)

Machine Translation code, can be replaced by other models.

In [121]:
def translate_text(prompt):

    client = OpenAI(api_key= OPENAI_API_KEY,)
    print(prompt)
    response = client.chat.completions.create(
      messages=[{
            "role": "user",
            "content": prompt,
          }],
      model="gpt-3.5-turbo",)
    translation = response.choices[0].message.content.strip().split("\n")[0]
    return translation

Quality Estimation

In [122]:
def quality_estimator(original_text, translated_text):
  client = OpenAI(api_key= OPENAI_API_KEY,)
  prompt = f"Evaluate the quality estimation of the following source and translation sentence pairs by following a step-by-step process: \
    Step 1: Estimate the perplexity of the translated sentence.\
    Step 2: Determine the token-level similarity between the source and translatedsentences.\
    Step 3: Combine the results and classify the translation quality into one of the following categories:'No meaning preserved', 'Some meaning preserved, but not understandable', 'Some meaning preserved and understandable', 'Most meaningpreserved, minor issues',or 'Perfect translation'.\
    Source:{original_text}.Translation:{translated_text}"
  print(prompt)
  response = client.chat.completions.create(
    messages=[{
          "role": "user",
          "content": prompt,
        }],
    model="gpt-3.5-turbo",
    )
  result = response.choices[0].message.content
  return result

## Baseline Test

In [123]:
# Devtest folder will be used for this baseline testing as dev
# may be more likely to appear in training data
flores_dataset = "../flores/floresp-v2.0-rc.2/devtest"

language_datasets = os.listdir(flores_dataset)

## Select Source & Target Languages

In [124]:

def select_random_language_from_dataset():


    # Randomly select source and target languages
    src_language_dataset = random.choice(language_datasets)
    targ_language_dataset = random.choice(language_datasets)

    # Assure that the source and target languages are not the same
    while targ_language_dataset == src_language_dataset:
        targ_language_dataset = random.choice(language_datasets)

    # Pull a random line from source and target datasets

    src_sentences = []
    targ_sentences = []

    def read_dataset(path):
        with open(flores_dataset + "/" + path, 'r', encoding="utf-8") as dataset_file:
            lines = dataset_file.readlines()
            total_lines = len(lines)
        return lines, total_lines

            
    src_lines, total_lines = read_dataset(src_language_dataset)
    targ_lines, total_lines = read_dataset(targ_language_dataset)

    selected_line_int = random.randint(1, total_lines)


    selected_line_src = src_lines[selected_line_int - 1]
    selected_line_targ = targ_lines[selected_line_int - 1]

    return selected_line_src, selected_line_targ, src_language_dataset, targ_language_dataset

    # print(selected_line_int)
    # print(selected_line_src)

    # print(src_sentences)
    # print(targ_sentences)


In [125]:
# Translate source to target using our POC


In [126]:
# Translate source to target using stock GPT 3.5 Turbo

def translate_gpt_api(selected_line_src, src_language_dataset, targ_language_dataset):
    prompt = f'Translate "{selected_line_src}" from the {src_language_dataset} language into the target language {targ_language_dataset}. Output only the translated sentence'
    translation = ""

    client = OpenAI(api_key= OPENAI_API_KEY,)
    print(prompt)
    response = client.chat.completions.create(
        messages=[{
            "role": "user",
            "content": prompt,
            }],
        model="gpt-3.5-turbo",)
        # model="gpt-4o",)

    hypothesized_translation = response.choices[0].message.content.strip().split("\n")[0]

    # print("\n\nMANUAL CHECK LIST PARAMS\n\n")

    # print(src_language_dataset, src_sentences)
    # print(targ_language_dataset, targ_sentences)


    # print(f'TRANSLATION: {hypothesized_translation}')

    return hypothesized_translation



## Compute spBLEU & chrF++ Scores

Below are reference cells using the scores with english language sentence showing how it works. 

```hypothesized_translation``` can be just a string which contains the translation from our model

```target_translation``` should be a list of strings (the score expects list of lists of strings)

---

All sentences being fed to the score, ```hypothesized_translation``` and all strings in ```target_translation``` are all in the same language. Be careful with what each variable contains there are similar named variables in this notebook.

In [127]:
def get_scores(hypothesized_translation, targ_selected_line, target_translation):
    
    print('\n\n')
    
    bleu_score  = sacrebleu.corpus_bleu([hypothesized_translation], [target_translation], tokenize="intl")
    print("spBLEU Score: ", bleu_score.score)

    chr_score  = sacrebleu.corpus_chrf([hypothesized_translation], [target_translation])

    print("chrF++ Score: ", chr_score.score)

    print("\n")

    print("Hypothesized Translation: ", hypothesized_translation)
    print("Target Translation: ", targ_selected_line)

    # return bleu_score, chr_score

## GPT 3.5 Turbo Base

In [130]:
src_selected_line, targ_selected_line, src_language_dataset, targ_language_dataset = select_random_language_from_dataset()


hypothesized_translation = translate_gpt_api(src_selected_line, src_language_dataset, targ_language_dataset)
target_translation = [targ_selected_line]

get_scores(hypothesized_translation, targ_selected_line, target_translation)
print(targ_selected_line)

Translate "Ko tshimologong, lenaneo leno le ne le gasiwa fela mo tsebeng ya nthanete ya  seromanowa sa Toginet, tsebe e e ikemeseditseng go gasa puisano ya diromamowa.
" from the devtest.tsn_Latn language into the target language devtest.spa_Latn. Output only the translated sentence



spBLEU Score:  3.4784906414591865
chrF++ Score:  28.406866326347696


Hypothesized Translation:  Cuando Tshimologong, este espacio solo está reservado para el programa de fin de semana de Toginet, donde se concentra en la mezcla de música.
Target Translation:  En sus inicios, el programa se emitía solo en el sitio de radio por internet, TogiNet Radio, un espacio de larga data enfocado en el formato de radio hablada.

En sus inicios, el programa se emitía solo en el sitio de radio por internet, TogiNet Radio, un espacio de larga data enfocado en el formato de radio hablada.



## Our POC

In [131]:
src_selected_line, targ_selected_line, src_language_dataset, targ_language_dataset = select_random_language_from_dataset()

In [133]:
poc_prompt = prompt_generator(src_selected_line, src_language_dataset, targ_language_dataset)

poc_hypothesized_translation = translate_text(poc_prompt)


# Scores
target_translation = [targ_selected_line]

get_scores(poc_hypothesized_translation, targ_selected_line, target_translation)

Translate the following text from mar_Deva into cym_Latn: आपण केवळ शिपबोर्ड सहल वापरुन किनारपट्टीवर जात असल्यास आपल्याला वेगळ्या व्हिजाची (2009 पर्यंत) आवश्यकता नाही.





spBLEU Score:  6.354504873535819
chrF++ Score:  25.087676048724433


Hypothesized Translation:  Os nad ydi dim ond ar y long ar gyfer neidio i'r arfordir, nid oes angen ffagiau gwahanol arnoch hyd at 2009.
Target Translation:  Os byddwch chi ond yn mynd i'r lan gan ddefnyddio gwibdeithiau bwrdd llong ni fydd angen fisa ar wahân arnoch chi (yn 2009).



At once

In [143]:
src_selected_line, targ_selected_line, src_language_dataset, targ_language_dataset = select_random_language_from_dataset()


hypothesized_translation = translate_gpt_api(src_selected_line, src_language_dataset, targ_language_dataset)
target_translation = [targ_selected_line]

get_scores(hypothesized_translation, targ_selected_line, target_translation)
print(targ_selected_line)






poc_prompt = prompt_generator(src_selected_line, src_language_dataset, targ_language_dataset)

poc_hypothesized_translation = translate_text(poc_prompt)


# Scores
target_translation = [targ_selected_line]

get_scores(poc_hypothesized_translation, targ_selected_line, target_translation)

Translate "Molao wa Fora ba dikgetho o hlakisa ka tiyo tshebetso ena.
" from the devtest.sot_Latn language into the target language devtest.smo_Latn. Output only the translated sentence



spBLEU Score:  4.724932626401583
chrF++ Score:  17.85745914348874


Hypothesized Translation:  O le Molao o lo'o fa'amalie i le faia o ni ti'o lava i lelei.
Target Translation:  O le tulafono o palota fa’a-Farani e fa’amalosia ai le vaevaeina o taualumaga.

O le tulafono o palota fa’a-Farani e fa’amalosia ai le vaevaeina o taualumaga.

Translate the following text from sot_Latn into smo_Latn: Molao wa Fora ba dikgetho o hlakisa ka tiyo tshebetso ena.





spBLEU Score:  2.2796871594840864
chrF++ Score:  11.883268435023803


Hypothesized Translation:  The work of Fora is not a mere decision but rather an enhancement of this service.
Target Translation:  O le tulafono o palota fa’a-Farani e fa’amalosia ai le vaevaeina o taualumaga.

